# An Example Physics Analysis

This notebook is an example of what post-Spyral analysis might look like. To use this notebook all four phases of Spyral *must* have been run on the data. After all of the phases are run on a dataset, it is time to do some actual physics analysis! In general physics analysis is highly specific to the experiment being run, but this notebook aims to give a little overview on what an analysis might look like if one wants to use some of the tools given by the application. 

First we import our modules we need. We'll also define a helpful constant, which we'll use to convert $B\rho$ to momentum.

In [ ]:
from spyral.core.constants import AMU_2_MEV, QBRHO_2_P

from spyral_utils.nuclear import NuclearDataMap
from spyral_utils.nuclear.target import GasTarget, load_target
from spyral_utils.plot import Histogrammer

import polars as pl
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import vector

%matplotlib widget
plt.close()


Now we'll load our configuration. The configuration will help us define the target material (the active target) which we'll use for some energy loss analysis. We'll also use the nuclear data map to get the masses for the reactants we're interested in. This example loads the data for the reaction $^{16}\mathrm{C}\left(\mathrm{d}, \mathrm{d}\right){}^{16}\mathrm{C}$ in inverse kinematics (where $^{16}\mathrm{C}$ is the beam projectile). We also define the projectile starting energy, that is the energy from the accelerator after passing through the window foils of the AT-TPC.

In [ ]:
# Set some parameters
# Here we won't use Spyral itself, as this example is supposed to show
# what the post-Spyral analysis should look like
solver_result_path = Path("/path/to/your/workspace/InterpSolver/")
target_material_path = Path("/path/to/your/target/material/file.json")

# Run number range (inclusive)
run_min = 16
run_max = 16

# Specify your nuclei

# The nucleus we observe (the one we fitted)
ejectile_z = 1
ejectile_a = 2

# The incoming nucleus (the beam)
projectile_z = 6
projectile_a = 16

# The target nucleus
target_z = 1
target_a = 2

residual_z = target_z + projectile_z - ejectile_z
residual_a = target_a + projectile_a - ejectile_a

if residual_z < 0:
    raise Exception(f"Illegal nuclei! Residual Z: {residual_z}")
if residual_a < 1:
    raise Exception(f"Illegal nuclei! Residual A: {residual_a}")


Setup our nuclear data objects

In [ ]:
nuclear_map = NuclearDataMap()

target_material = load_target(target_material_path, nuclear_map)
if not isinstance(target_material, GasTarget):
    print('Target error!')

ejectile = nuclear_map.get_data(ejectile_z, ejectile_a)
projectile = nuclear_map.get_data(projectile_z, projectile_a)
target = nuclear_map.get_data(target_z, target_a)
residual = nuclear_map.get_data(residual_z, residual_a)

# Initial beam energy
mass_amu = projectile.mass / AMU_2_MEV
proj_energy_start = 11.5 * mass_amu #MeV


Now we'll create a histogramming system. The Histogrammer class wraps the numpy histogramming functionality. This is useful because we'll probably want to look at several runs, and neither numpy or matplotlib provide clean native functionality for this. We'll define a couple of histograms that will plot some useful data for us. 

In [ ]:

grammer = Histogrammer()
grammer.add_hist2d('ke_theta', (360, 1600), ((0.0, 180.0), (0.0, 80.0)))
grammer.add_hist2d('ke_phi', (360, 1600), ((0.0, 360.0), (0.0, 80.0)))
grammer.add_hist1d('ex', 250, (-5.0, 10.0))
grammer.add_hist1d('chisq', 1000, (0.0, 1.0e-4))

Now we'll do some data analysis! Here we're going to calculate the ejectile (detected) particle kinetic energy from our fitted $B\rho$ and then combine that with the projectile kinetic energy, polar reaction angle, and masses to calculate the excitation energy of the residual nucleus. Note that we use the vertex position to calculate the distance travelled for the beam in the gas and then calculate the energy lost by the beam travelling to the reaction vertex. This data is then given to the histogrammer.

In [ ]:
kinetics = np.empty(0, float)
angles = np.empty(0, float)
chisq = np.empty(0, float)
target_vector = vector.array({"px": [0.0], "py": [0.0], "pz": [0.0], "E": [target.mass]})
for run in range(run_min, run_max+1):
    df = None
    try:
        path = solver_result_path / f"run_{run:04d}_{ejectile.isotopic_symbol}.parquet"
        df = pl.read_parquet(path)
    except Exception:
        continue


    # df = df.filter((pl.col('redchisq') < 1.0e-4) & (pl.col("vertex_z") > 0.005) & (pl.col("vertex_z") < 0.995))
    df = df.filter((pl.col("vertex_z") > 0.005) & (pl.col("vertex_z") < 0.995))

    # Construct the projectile vectors (beam)
    vertices = df.select(['vertex_x', 'vertex_y', 'vertex_z']).to_numpy()
    distances = np.linalg.norm(vertices, axis=1)
    projectile_ke = proj_energy_start - target_material.get_energy_loss(projectile, proj_energy_start, distances)
    projectile_vector = vector.array({
        "px": np.zeros(len(projectile_ke)),
        "py": np.zeros(len(projectile_ke)),
        "pz": np.sqrt(projectile_ke * (projectile_ke + 2.0 * projectile.mass)),
        "E": projectile_ke + projectile.mass
    })

    # Construct the ejectile vectors (detected)
    brho = df.select('brho').to_numpy().flatten()
    momentum = df.select('brho').to_numpy().flatten() * float(ejectile.Z) * QBRHO_2_P
    kinetic_energy = np.sqrt(momentum**2.0 + ejectile.mass**2.0) - ejectile.mass
    polar = df.select('polar').to_numpy().flatten()
    az = df.select('azimuthal').to_numpy().flatten()
    cs = df.select('redchisq').to_numpy().flatten()
    ejectile_vector = vector.array({
        "px": momentum * np.sin(polar) * np.cos(az),
        "py": momentum * np.sin(polar) * np.sin(az),
        "pz": momentum * np.cos(polar),
        "E": np.sqrt(momentum**2.0 + ejectile.mass**2.0)
    })

    # Do the kinematics
    residual_vector = target_vector + projectile_vector - ejectile_vector # type: ignore

    grammer.fill_hist2d('ke_theta', np.rad2deg(polar), kinetic_energy)
    grammer.fill_hist2d('ke_theta_resid', np.rad2deg(polar), kinetic_energy)
    grammer.fill_hist2d('ke_phi', np.rad2deg(az), kinetic_energy)
    grammer.fill_hist1d('ex', residual_vector.mass - residual.mass) # Excitation energy is "extra" mass
    grammer.fill_hist1d("chisq", cs)
    kinetics = np.append(kinetics, kinetic_energy)
    angles = np.append(angles, np.rad2deg(polar))
    chisq = np.append(chisq, cs)
print(f"Median chisq: {np.median(chisq)}")

We can also compare the kinematics to the expected values. Here we'll calculate the ejectile kinetic energy for a range of reaction angles. Note that in this case we cannot account for the exact beam energy after travelling through the gas without doing a form of Monte-Carlo simulation. By default we only calculate the ground state of the resiudal.

In [ ]:
angle_range = np.linspace(0., np.pi, 1000)

ex = 0.0

Q = projectile.mass + target.mass - (ejectile.mass + residual.mass + ex)
term1 = np.sqrt(projectile.mass * ejectile.mass * proj_energy_start) / (ejectile.mass + residual.mass) * np.cos(angle_range)
term2 = (proj_energy_start * (residual.mass - projectile.mass) + residual.mass*Q) / (residual.mass + ejectile.mass)
eject_energy = term1 + np.sqrt(term1*term1 + term2)
eject_energy = eject_energy**2.0

Now lets make some plots! These first ones will be the energy-angle relationship for the ejectile (detected) particle. The interesting one is energy vs. polar ($\theta$). On this histogram we also plot our calculated energy angle relationship. Here we're looking for general agreement with the trend, not exact values. Because the calculation doesn't have a good way to handle the beam energy loss, the data will always look a little more "smeared" than the calculation would expect.

In [ ]:
ke_theta = grammer.get_hist2d("ke_theta")
ke_phi = grammer.get_hist2d("ke_phi")
fig, ax = plt.subplots(1,2)
mesh = ax[0].pcolormesh(ke_theta.x_bins, ke_theta.y_bins, ke_theta.counts, norm="log")
ax[0].set_title("Kinetic Energy vs. Polar Angle")
ax[0].set_xlabel(r"$\theta$ (deg)")
ax[0].set_ylabel("Kinetic Energy (MeV)")
fig.colorbar(mesh, ax=ax[0])
ax[0].plot(np.rad2deg(angle_range), eject_energy)
mesh = ax[1].pcolormesh(ke_phi.x_bins, ke_phi.y_bins, ke_phi.counts)
ax[1].set_title("Kinetic Energy vs. Azimuthal Angle")
ax[1].set_xlabel(r"$\phi$ (deg)")
ax[1].set_ylabel("Kinetic Energy (MeV)")
fig.colorbar(mesh, ax=ax[1])
fig.set_figheight(8.0)
fig.set_figwidth(11.0)


We'll also plot our calculated residual excitation energy. In many ways this is the primary plot of interest, as it should show you the states in the residual that were populated in the reaction with their relative intensities (uncorrected for detector efficiency of course).

In [ ]:
ex_hist = grammer.get_hist1d("ex")
fig, ax = plt.subplots(1,1)
ax.stairs(ex_hist.counts, edges=ex_hist.bins)
ax.set_title(f"{residual.get_latex_rep()} Spectrum")
ax.set_xlabel("Excitation Energy (MeV)")
ax.set_ylabel("Counts")
fig.set_figwidth(8.0)

If the data you see doesn't match your expectations, there probably was an issue with one of the earlier analysis phases or the configuration parameters. Try  tweaking some the configuration parameters and see what it does to the final results! Below is also a plot of the error (chisquare) for your data.

In [ ]:
hist = grammer.get_hist1d("chisq")
fig, ax = plt.subplots(1,1)
ax.stairs(hist.counts, edges=hist.bins)
ax.set_title(f"{residual.get_latex_rep()} Error")
ax.set_xlabel("Error")
ax.set_ylabel("Counts")
fig.set_figwidth(8.0)

Finally, below we demonstrate how to fit your excitation energy spectrum and extract some state parameters. This *will not* work by default, it was tuned to a random dataset. But it will demonstrate how to use lmfit to fit your spectrum

In [ ]:
import lmfit

hist = grammer.get_hist1d("ex")

peak1 = lmfit.models.GaussianModel(prefix="peak1_")
peak1_bins, peak1_counts = hist.get_subrange((-0.5, 0.5))
peak1_bins += 0.5 * hist.bin_width
peak1_weights = 1.0/np.sqrt(peak1_counts)
peak1_weights[peak1_counts == 0.0] = 1.0
pars = peak1.guess(x=peak1_bins, data=peak1_counts, weights=peak1_weights)

# peak2 = lmfit.models.LorentzianModel(prefix="peak2_")
# peak2_bins, peak2_counts = hist.get_subrange((0.0, 1.0))
# peak2_bins += 0.5 * hist.bin_width
# peak2_weights = 1.0/np.sqrt(peak2_counts)
# peak2_weights[peak2_counts == 0.0] = 1.0
# pars.update(peak2.guess(x=peak2_bins, data=peak2_counts, weights=peak2_weights))

# peak3 = lmfit.models.LorentzianModel(prefix="peak3_")
# peak3_bins, peak3_counts = hist.get_subrange((2.0, 3.0))
# peak3_bins += 0.5 * hist.bin_width
# peak3_weights = 1.0/np.sqrt(peak3_counts)
# peak3_weights[peak3_counts == 0.0] = 1.0
# pars.update(peak3.guess(x=peak3_bins, data=peak3_counts, weights=peak3_weights))

# peak4 = lmfit.models.LorentzianModel(prefix="peak4_")
# peak4_bins, peak4_counts = hist.get_subrange((3.0, 5.0))
# peak4_bins += 0.5 * hist.bin_width
# peak4_weights = 1.0/np.sqrt(peak4_counts)
# peak4_weights[peak4_counts == 0.0] = 1.0
# pars.update(peak4.guess(x=peak4_bins, data=peak4_counts, weights=peak4_weights))

# peak5 = lmfit.models.LorentzianModel(prefix="peak5_")
# peak5_bins, peak5_counts = hist.get_subrange((5.0, 6.5))
# peak5_bins += 0.5 * hist.bin_width
# peak5_weights = 1.0/np.sqrt(peak5_counts)
# peak5_weights[peak5_counts == 0.0] = 1.0
# pars.update(peak5.guess(x=peak5_bins, data=peak5_counts, weights=peak5_weights))

# bkgnd = lmfit.models.LinearModel(prefix="bkgnd_")
# bkgnd_bins, bkgnd_counts = hist.get_subrange((-2.0, -1.0))
# bkgnd_bins += 0.5 * hist.bin_width
# bkgnd_weights = 1.0/np.sqrt(bkgnd_counts)
# bkgnd_weights[bkgnd_counts == 0.0] = 1.0
# pars.update(bkgnd.guess(x=bkgnd_bins, data=bkgnd_counts))
# pars["bkgnd_slope"].min = 0.0

# total_fit = peak1 + peak2 + peak3 + peak4 + peak5 + bkgnd
total_fit = peak1
total_bins, total_counts = hist.get_subrange((-1.0, 1.0))
total_bins += 0.5 * hist.bin_width
total_weights = 1.0/np.sqrt(total_counts)
total_weights[total_counts == 0.0] = 1.0
total_result = total_fit.fit(params=pars, x=total_bins, data=total_counts, weights=total_weights)
comps = total_result.eval_components(x=total_bins)
total_result


In [ ]:
fig, ax = plt.subplots(1,1)
ax.stairs(hist.counts, edges=hist.bins, label="Spectrum")
ax.plot(total_bins, total_result.best_fit, label="Total Fit")
ax.plot(total_bins, comps["peak1_"], label="peak1")
# ax.plot(total_bins, comps["peak2_"], label="peak2")
# ax.plot(total_bins, comps["peak3_"], label="peak3")
# ax.plot(total_bins, comps["peak4_"], label="peak4")
# ax.plot(total_bins, comps["peak5_"], label="peak5")
# ax.plot(total_bins, comps["bkgnd_"], label="bkgnd")
ax.set_title(f"{residual.get_latex_rep()} Spectrum Fit")
ax.set_xlabel("Excitation Energy (MeV)")
ax.set_ylabel("Counts")
fig.set_figwidth(8.0)
fig.legend()